# Hakai Nutrient QA-QC 
---
# Start Notebook
## Install packages 
Google colab servers has already a few commonly used packages installed. To install those missing, which are specific toh Hakai let's run the following commands:

In [1]:
!pip install hakai-api
!pip install git+git://github.com/HakaiInstitute/ioos_qc.git@development
!pip install git+git://github.com/HakaiInstitute/process_ocean_timeseries

  Cloning git://github.com/HakaiInstitute/ioos_qc.git (to revision development) to /tmp/pip-req-build-ehrweks3
  Running command git clone -q git://github.com/HakaiInstitute/ioos_qc.git /tmp/pip-req-build-ehrweks3
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
     |████████████████████████████████| 1.0MB 35.2MB/s 
     |████████████████████████████████| 112kB 45.0MB/s 
     |████████████████████████████████| 133kB 46.7MB/s 
     |████████████████████████████████| 15.4MB 202kB/s 
     |████████████████████████████████| 6.6MB 27.1MB/s 
     |████████████████████████████████| 552kB 31.9MB/s 
  Created wheel for ioos-qc: filename=ioos_qc-1.0.0-cp37-none-any.whl size=44554 sha256=1d7a029fb18421b97c6c9d1c8b0c87c96b6971a5b1154a5150eb6d2e82868595
  Stored in directory: /tmp/pip-ephem-wheel-cache-8preaxkt/wheels/2f/bb/7b/19f2997af03515525ae518c267

## Import the python packages needed

In [2]:
# Let's load pandas for working with the data in table
import pandas as pd 
import numpy as np

# Let's load seaborn to plot the data
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# This to install the hakai api tool and be able to download some data from hakai's database
from hakai_api import Client

# Install ioos_qc which is used to qc data
from ioos_qc.config import QcConfig
from ioos_qc import qartod

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets, HBox, VBox
import time

from process_ocean_data.tools import qc as review

# Define some project standards

In [3]:
# Flag convention
flag_convention = 'HAKAI'
flag_color_map = {key:value['Color'] for key,value in review.flag_conventions[flag_convention].items()}

# And the interesting variables to use from the Hakai database
nutrient_variables = ['no2_no3_um','sio2','po4']

pd.options.display.max_columns = 100

# Download data from the Hakai database
For more information regarding the Hakai API, go [here](https://github.com/HakaiInstitute/hakai-api).

You can find a list of all the data type endpoints [here](http://hakaiinstitute.github.io/hakai-api/#endpoints).

In [42]:
# Let's retrieve the endpoint to retrieve nutrients data:
endpointUrl = '/eims/views/output/nutrients'
site_id = 'DFO2'
start_time = '2012-01-01'
end_time = '2022-06-11'
# We'll retrieve data only associated with QU39 between January 1st 2019 to January 1st 2020
filterUrl = 'site_id={0}&collected>{1}&collected<{2}&limit=-1'.format(
    site_id, start_time, end_time
)

# Get Hakai Data    
#Get Data from Hakai API
client = Client() # Follow stdout prompts to get an API token

# Make a data request for sampling stations
url = '%s/%s?%s' % (client.api_root,endpointUrl,filterUrl)
response = client.get(url)
df = pd.DataFrame(response.json())
original_columns = df.columns
print(str(len(df))+' records downloaded')
df.head()

810 records downloaded


,action,event_pk,rn,is_replicate,date,work_area,organization,survey,sampling_bout,site_id,project_specific_id,hakai_id,source,lat,long,gather_lat,gather_long,collection_method,line_out_depth,pressure_transducer_depth,filtered,filter_type,volume,installed,collected,preserved,analyzed,lab_technician,nh4_,no2_no3_um,no2_no3_ugl,no2_no3_units,tp,tdp,tn,tdn,srp,po4,sio2,po4pfilt,no3nfilt,po4punfl,no3nunfl,nh4nunfl,nh4__flag,no2_no3_flag,tp_flag,tdp_flag,tn_flag,tdn_flag,srp_flag,po4_flag,sio2_flag,po4pfilt_flag,no3nfilt_flag,po4punfl_flag,no3nunfl_flag,nh4nunfl_flag,analyzing_lab,row_flag,metadata_qc_flag,quality_level,comments,quality_log
0,,7209,1,None,2013-04-08,CALVERT,HAKAI,"RVRS,HSVY",1,DFO2,71,NUT71,M,51.5208,-127.5583,NaN,NaN,None,5,NaN,None,0.45nm,13,None,2013-04-08T21:30:00.000Z,2013-04-08T07:00:00.000Z,None,None,None,9.106,None,uM,None,None,None,None,None,0.787,19.567,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Metadata has been QCd by BF\r2: this sample...
1,,7209,2,None,2013-04-08,CALVERT,HAKAI,"RVRS,HSVY",1,DFO2,72,NUT72,M,51.5208,-127.5583,NaN,NaN,None,5,NaN,None,0.45nm,13,None,2013-04-08T21:30:00.000Z,2013-04-08T07:00:00.000Z,None,None,None,NaN,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Available,None,Technicianm,,1: Metadata has been QCd by BF | sample was ta...
2,,7209,1,None,2013-04-08,CALVERT,HAKAI,"RVRS,HSVY",1,DFO2,73,NUT73,M,51.5208,-127.5583,NaN,NaN,None,30,NaN,None,0.45nm,13,None,2013-04-08T21:30:00.000Z,2013-04-08T07:00:00.000Z,None,None,None,19.211,None,uM,None,None,None,None,None,1.674,34.327,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Metadata has been QCd by BF\r2: this sample...
3,,7209,2,None,2013-04-08,CALVERT,HAKAI,"RVRS,HSVY",1,DFO2,74,NUT74,M,51.5208,-127.5583,NaN,NaN,None,30,NaN,None,0.45nm,13,None,2013-04-08T21:30:00.000Z,2013-04-08T07:00:00.000Z,None,None,None,NaN,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Available,None,Technicianm,,1: Metadata has been QCd by BF | sample was ta...
4,,7275,1,None,2013-05-10,CALVERT,HAKAI,RVRS,1,DFO2,301,NUT301,M,51.5208,-127.5583,NaN,NaN,None,5,NaN,None,0.45nm,13,None,2013-05-10T18:56:00.000Z,2013-05-10T07:00:00.000Z,None,None,None,7.254,None,uM,None,None,None,None,None,0.606,15.318,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Metadata has been QCd by BF


In [43]:
# Let's convert the collected time to a datetime object variable called  time 
#  and extract the from those datetime objects the year and month 
df['time'] = pd.to_datetime(df['collected'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayoftheyear'] = pd.to_timedelta(df['time'].dt.dayofyear, unit='d')

# Define a depth variable which is: 
#   - pressure_transducer_depth (if available)
#   - OR line_out_depth
df['depth'] = df['pressure_transducer_depth'].fillna(df['line_out_depth'])

# Review Replicates
## Pool Standard Deviation


In [44]:
 # Let's create a pooled standard deviation function
def pooled_standard_deviation(df_to_review,count_col='count',std_col='std'):
    # Keep only records that have replicates
    df_replicates = df_to_review[df_to_review[count_col]>1]
    upper =  df_replicates[count_col].sub(-1).mul(df_replicates[std_col].pow(2)).sum()
    lower = df_replicates[count_col].sub(-1).sum()
    pooled_std = np.sqrt(upper/lower)
    return pooled_std

In [45]:
df_grouped = df.groupby(['site_id','line_out_depth','collected']).agg(['mean','std','count'])

for var in nutrient_variables:
    pool_std = pooled_standard_deviation(df_grouped[var])
    print('{0} pool.std.: {1}'.format(var,pool_std))

no2_no3_um pool.std.: 0.30754398572835884
sio2 pool.std.: 0.6081076927376894
po4 pool.std.: 0.07335183082331137


## Replicates Standard Deviation Distribution
Present the distribution of the standard deviations the replicate samples for each line_out_depth.

In [46]:
px.histogram(df_grouped['po4'].reset_index().dropna(subset=['std']).sort_values('line_out_depth'),
             x='std',color='line_out_depth', hover_name='collected',
             marginal='box')

# Apply detection limit flag
Samples with values lower than the dectection limit will be flag as BDL.



In [47]:
# If lower than detection limit, flag as BDL
df.loc[df['no2_no3_um']<0.036 ,'no2_no3_um_bdl_flag']='BDL'
df.loc[df['po4']<0.032 ,'po4_bdl_flag']='BDL'
df.loc[df['sio2']<0.1 ,'sio2_bdl_flag']='BDL'

# Time series QARTOD tests
Review each depth time series for a station and run timeseries specific test on them. The configuarion dictionary below, list of each variables and depth ranges the tests that will be applied.


## Set QARTOD Tests Configuration

In [48]:
qc_config = [{
    "depth_range":{
        "minimum":-5,
        "maximum":55
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "aggregate": {}
            }
                    
        }
    }
},{
    "depth_range":{
        "minimum":55,
        "maximum":300
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "spike_test": {
                    "suspect_threshold": 2,
                    "fail_threshold": 3,
                    "method": "differential",
                    "n_dev_suspect":1,
                    "n_dev_fail":2,
                    "n_records":40,
                    "min_records":20
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "spike_test": {
                    "suspect_threshold": 0.5,
                    "fail_threshold": 1,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":50,
                    "min_records":40
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "spike_test": {
                    "suspect_threshold":8,
                    "fail_threshold": 12,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":30,
                    "min_records":20
                    },
                "aggregate": {}
            }
                    
        }
    }
}]


## Run QARTOD Tests

In [49]:
# Run QARTOD tests
# We are using the deprecated QcConfig method and hopefully will move 
#  to a new stream method soon.
time = 'time'
depth = 'depth'
group_timeseries = ['line_out_depth']
for item in qc_config:
    df_depth_range = df[(df['depth']>item['depth_range']["minimum"]) & \
                        (df['depth']<item['depth_range']["maximum"])]
    for line_out_depth, timeserie in df_depth_range.groupby(['site_id','line_out_depth']):
        timeserie = timeserie.sort_values(time)
        for var in item['streams'].keys():
            qc = QcConfig(item['streams'][var])
            qc_result = qc.run(
                inp=timeserie[var],
                tinp=timeserie[time],
                zinp=timeserie[depth],
            )
            for module,tests in qc_result.items():
                for test, flag in tests.items():
                    df.loc[timeserie.index, var+'_'+module+"_"+test] = flag

# Map QARTOD Flags to Hakai Convention
for var in df.filter(like='qartod').columns:
     df[var] = df[var].replace({pd.NA:9}).astype(int).replace(review.flag_conventions['mapping']['QARTOD-HAKAI'])

## Review QARTOD Results

In [50]:
flag_color_map = {key:value['Color'] for key,value in review.flag_conventions['HAKAI'].items()}

var = 'sio2'
line_out_depth= [100]
flag_var = df.filter(regex='flag|qartod').columns.tolist()
px.scatter(df[df['line_out_depth'].isin(line_out_depth)].sort_values(['line_out_depth',var+'_qartod_aggregate']),
             x='time',y=var,
             color=var+'_qartod_aggregate', 
             hover_data=['hakai_id']+flag_var,
             color_discrete_map=flag_color_map)

# Red field ratio

In [51]:
max_depth = 100
figs=px.scatter(df[df['line_out_depth']<max_depth],
               x='po4',
               y='no2_no3_um',
               color='line_out_depth', 
               hover_data=['hakai_id','date'], 
               template='simple_white',
               title='PO4',
               labels={'po4':'PO4 (uM)', 'line_out_depth':'Bottle Target Depth (m)'},facet_col='year')
kk=1
for item in figs.data:
  figs.add_trace(go.Scatter(x = [0, 3],
                            y =  [0, 48],
                            mode='lines',
                            line_color='red',
                            showlegend=False
                            ),
                row=1,
                col=kk)
  kk+=1
figs.show()

# Review Interannual Variability
Let's compute the average value measured for each depth and the associated standard deviation.

## Compute the seasonal variability
### Monthly Values


In [52]:
# Compute the monthly value recorded for each station and line_out_depth 
df_seasonal = df.groupby(['site_id','line_out_depth','month']) \
  .agg(['mean','std'])[nutrient_variables] \
  .reset_index()
df_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_seasonal.columns.values]

# Add a seasonal value to each data
df_with_seasons = pd.merge(df,df_seasonal,on=['site_id','line_out_depth','month'], suffixes=('','_seasonal'))


### Over 60 days window

In [95]:
# TODO not that pandas.rolling isn't compatible with center window yet.
# Or get the season with a 30 days running window instead
window = '30d'
df_running_seasonal = df.groupby(['site_id','line_out_depth']).apply(
    lambda x: x.sort_values('dayoftheyear').set_index('dayoftheyear')[nutrient_variables]\
       .rolling(window).agg(['mean','std']).reset_index()
)
df_running_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_running_seasonal.columns.values]

# Since rolling is looking the days prior, we'll center the value to the middle of the window.
#   rolling(center) isn't yet available for datetime index
df_running_seasonal['dayoftheyear'] = df_running_seasonal['dayoftheyear']-pd.to_timedelta(window)/2
df_running_seasonal.loc[df_running_seasonal['dayoftheyear']<pd.to_timedelta('0d'),'dayoftheyear'] += pd.to_timedelta('365d')

# Clean the matrix a bit
df_running_seasonal = df_running_seasonal.reset_index()\
    .drop('level_2',axis='columns')\
    .set_index(['site_id','line_out_depth','dayoftheyear'])\
    .add_suffix('_seasonal')




In [127]:
# TODO Improve this section which is not working 
seasonal_resampled = pd.DataFrame()
for index, item in df_running_seasonal.groupby(['site_id','line_out_depth']):
    item = item.resample('1D',level='dayoftheyear').mean().interpolate().reset_index()
    item['site_id'] = index[0]
    item['line_out_depth'] = index[1]
    seasonal_resampled.append(item)

In [129]:
# TODO Merge seasonal average back with the data

## Make Monthly Box Plots

In [81]:
review_depth = 20
var = 'sio2'

# Show the seasonality of the data
px.box(df.sort_values('line_out_depth'),x='month',y='no2_no3_um',
        color='line_out_depth', animation_frame='line_out_depth',
       hover_name='hakai_id')

In [82]:
# Review each depth timeseries and
review_depth = 0
alpha = 2
var = 'no2_no3_um'

#  compare seasonal average to value recorded
for var_name in ['no2_no3_um','po4','sio2']:
    df[var_name+'_seasonal_flag'] = 'AV'

    residual = (df[var_name]-df[var_name+'_mean_seasonal']).abs()
    df.loc[residual> alpha*df[var_name+'_std_seasonal'], var_name+'_seasonal_flag'] = 'SVC'

# Isolate the review depth
df_temp = df[df['line_out_depth']==review_depth].sort_values('time')

# Plot data
fig = px.scatter(df_temp,x='time',y=var,color=var+'_seasonal_flag', color_discrete_map=flag_color_map,
                 hover_name='hakai_id')

fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"],name='running mean',
              line = dict(color='red', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"]+alpha*df_temp[var+"_std_seasonal"],
                         name='running mean+{0}*std'.format(alpha),
              line = dict(color='black', width=2, dash='dot')))
fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"]-alpha*df_temp[var+"_std_seasonal"],
                         name='running mean-{0}*std'.format(alpha),
                         line = dict(color='black', width=2, dash='dot')))

fig.update_layout(coloraxis_showscale=False)
fig.show()

KeyError: ignored

# Create Aggregated Suggested Flag


In [83]:
flags_considered = ['_qartod_aggregate','_bdl_flag']
for var in nutrient_variables:
    flag_vars = [var+item for item in flags_considered]
    df[var+'_review_flag'] = \
       df[flag_vars].apply(lambda x: review.compare_flags(list(x),convention='HAKAI'),axis='columns')

# Update Original Flags
df['po4_flag'] = df['po4_flag'].fillna(df['po4_review_flag'])
df['sio2_flag'] = df['sio2_flag'].fillna(df['sio2_review_flag'])
df['no2_no3_flag'] = df['no2_no3_flag'].fillna(df['no2_no3_um_review_flag'])

# Review Results Manually

In [121]:
# Plot profile
group_profile_by = 'date'
profile_list = widgets.SelectMultiple(
    options=df[group_profile_by].sort_values().drop_duplicates(),
    value=(df[group_profile_by].iloc[0],),
    description="Select Profiles",
    disabled=False,
)
def make_profile_plot(profiles):
    # Filter only selected profiles
    df_selected = df[df['date'].isin(profiles)]
    df_temp = df[df['date'].isin(profile_list.value)].sort_values(['date','depth'])
    fig = []
    for ii in [0,1,2]:
        fig.append(px.line(df_temp,x=nutrient_variables[ii],y='depth',color='date',hover_data=['hakai_id']))

    subfig = make_subplots(rows=1, cols=3)
    kk=1
    for item in fig:
        for subitem in item.data:
            if kk>1:
                subitem.showlegend=False
            subfig.add_trace(subitem,row=1,col=kk)
        subfig.update_xaxes(title_text=nutrient_variables[kk-1], row=1, col=kk)
        kk += 1
    subfig.update_yaxes(title_text='Depth (m)', row=1, col=1)
    subfig.update_yaxes(autorange="reversed")
    subfig.update_xaxes(matches=None)
    subfig.update_traces(mode='markers+lines')
    return subfig

interact(make_profile_plot, profiles=profile_list)

interactive(children=(SelectMultiple(description='Select Profiles', index=(0,), options=('2013-04-08', '2013-0…

<function __main__.make_profile_plot>

In [123]:
group_timeseries_by = 'line_out_depth'
# Plot TimeSeries
timeseries_id = widgets.SelectMultiple(
    options=df[group_timeseries_by].sort_values().drop_duplicates().values,
    value=(df[group_timeseries_by].iloc[0],),
    description="Select "+group_timeseries_by,
    disabled=False,
)

nutrient_id = widgets.ToggleButtons(
    options=nutrient_variables,
    value=nutrient_variables[0],
    description="Select "+group_timeseries_by,
    disabled=False,
)
timeseries_id
def plot_timeseries(nutrient, profile):
    flag_columns = df.filter(regex= nutrient+'.*(flag|qartod)').columns
    df_temp = df[df['line_out_depth'].isin(timeseries_id.value)].sort_values('time').copy()
    df_temp[df_temp.filter(like='flag').columns] = df_temp.filter(like='flag').astype(str)
    df_good = df_temp.loc[df_temp[nutrient.replace('_um','')+'_flag']=='AV']
    fig = px.line(df_good,
            x='time',
            y=nutrient,
            line_dash='line_out_depth',
            color=nutrient+'_review_flag',
            color_discrete_map={value:item['Color'] for value, item in review.flag_conventions['HAKAI'].items()}
           )
    fig2 = px.scatter(df_temp,
            x='time',
            y=nutrient,
            color=nutrient+'_review_flag',
            hover_data=['date','hakai_id','depth']+list(flag_columns),
            color_discrete_map={value:item['Color'] for value, item in review.flag_conventions['HAKAI'].items()}
           )
    for item in fig2.data:
        fig.add_trace(item)
    return fig
    
interact(plot_timeseries,nutrient=nutrient_id, profile=timeseries_id)

interactive(children=(ToggleButtons(description='Select line_out_depth', options=('no2_no3_um', 'sio2', 'po4')…

<function __main__.plot_timeseries>

In [ ]:
# Show me the data flagged
df.loc[df[['no2_no3_flag','sio2_flag','po4_flag']].isin(['SVC','SVD','BDL']).any(axis='columns')][['date','hakai_id','no2_no3_flag','sio2_flag','po4_flag']]

,date,hakai_id,no2_no3_flag,sio2_flag,po4_flag
18,2016-04-13,NUT6807,AV,AV,BDL
43,2017-04-30,NUT8295,SVC,AV,AV
45,2017-07-22,NUT8796,AV,AV,BDL
54,2018-03-31,NUT9527,AV,AV,BDL
101,2020-06-06,NUT12664,AV,AV,BDL


In [ ]:
# Review QC Manually ( Not Compatible with Colab)
df['no2_no3'] = df['no2_no3_um']
review.manual_qc_interface(df, ['no2_no3','sio2','po4'], 'HAKAI' ,review_flag="_flag")


# Save Result to Hakai Portal Compatible Excel Format

In [86]:
df_output = df[original_columns]

# Rename Columns to Match Portal Output
# Drop unchanged rows

###ATTENTION### The output has not yet been tested with the Hakai API.
df_output.to_excel('Hakai_Nutrient_Revision_{0}.xlsx'.format(pd.Timestamp.now().isoformat().replace(':','')))


# Report Figures

## Contour plot

In [124]:

# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
var = 'no2_no3_um'
df_pivot = pd.pivot_table(df,values=var,index='line_out_depth',columns='date',aggfunc='mean').interpolate(axis='index',limit=4).sort_index(axis=0).sort_index(axis=1).interpolate(axis='columns',limit=3)
fig = go.Figure(data =
    go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
               colorbar=dict(title=var, titleside='right'),
               colorscale='RdYlGn',
               ncontours=10,
               contours_coloring='heatmap'
               #,connectgaps=True
              ))
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside',showline=True)
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

## Scatter with colorbar

In [125]:
## Scatter with colorbar
fig = px.scatter(df.dropna(subset=['no2_no3_um'],axis=0),x='time',y='depth',
                 color='no2_no3_um',color_continuous_scale='RdYlGn',
                 hover_name='hakai_id')
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside')
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

In [126]:
# Add scatter per depth
px.scatter(df.sort_values(['line_out_depth','time']),x='time',y='no2_no3_um',color='depth')